In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

The system cannot find the path specified.
tar: Error opening archive: Failed to open 'spark-3.0.0-bin-hadoop3.2.tgz'


In [1]:
!pip install pyspark

     -------------------------------------- 281.3/281.3 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 199.7/199.7 KB ? eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764027 sha256=0568cc58c2d67d405e8c62f83759a7a9ce8807f235221b95c4f4a6f11e3c6d24
  Stored in directory: c:\users\hoang\appdata\local\pip\cache\wheels\05\75\73\81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built pyspark


You should consider upgrading via the 'E:\app\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/MyDrive/DataScience_project/dataset'

/content/gdrive/MyDrive/DataScience_project/dataset


In [2]:
import findspark
findspark.init()

Exception: Unable to find py4j in /content/spark-3.0.0-bin-hadoop3.2\python, your SPARK_HOME may not be configured correctly

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col, explode

In [7]:
SparkContext.setSystemProperty('spark.executor.memory','12g')
sc = SparkContext(master='local', appName='Recommendation')

In [8]:
spark = SparkSession(sc)

In [12]:
data = spark.read.csv('Review_clean.csv',header=True,inferSchema=True)

In [13]:
data.show(5, truncate=True)

+-----------+----------+-----------------+------+------------------+--------------------+--------------------+
|customer_id|product_id|             name|rating|             title|             content|  review_information|
+-----------+----------+-----------------+------+------------------+--------------------+--------------------+
|     709310|  10001012| Lân Nguyễn Hoàng|     3|Ko dùng đc thẻ nhớ|Lúcđầu quên thông...|không thẻ_nhớ lúc...|
|   10701688|  10001012| Nguyễn Khánh Hòa|     5|   Cực kì hài lòng|Tiki giao hàng nh...|cực_kì hài_lòng h...|
|   11763074|  10001012|  Toàn Phạm Khánh|     5|   Cực kì hài lòng|chất lượng camera...|cực_kì hài_lòng c...|
|    9909549|  10001012|Nguyen Quang Minh|     5|      Rất hài lòng|Hàng được đóng gó...|hài_lòng hàng đón...|
|    1827148|  10001012|      Phạm Bá Đức|     5|   Cực kì hài lòng|dễ cài đặt, chất ...|cực_kì hài_lòng c...|
+-----------+----------+-----------------+------+------------------+--------------------+--------------------+
o

In [14]:
data_sub = data.select(['product_id','rating','customer_id'])

In [15]:
data_sub.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- customer_id: string (nullable = true)



In [16]:
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import isnan, when, count, col, udf

In [17]:
data_sub = data_sub.withColumn('rating', data_sub["rating"].cast(DoubleType()))
data_sub = data_sub.withColumn('customer_id', data_sub["customer_id"].cast(IntegerType()))
data_sub = data_sub.withColumn('product_id', data_sub["product_id"].cast(IntegerType()))

In [18]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in data_sub.columns]).toPandas().T

,0
product_id,1656
rating,1718
customer_id,1687


In [19]:
data_sub = data_sub.na.drop(how='any')

In [20]:
data_sub.count()

361720

In [21]:
# Distinct users and products
users = data_sub.select("customer_id").distinct().count()
products = data_sub.select("product_id").distinct().count()
numerator = data_sub.count()

In [22]:
display(numerator, users, products)

361720

251467

4218

In [23]:
# Number of rating matrix could contain if no empty cells
denominator = users * products
denominator

1060687806

In [24]:
sparsity = 1- (numerator * 1.0/ denominator)
print("sparsity:"), sparsity

sparsity:


(None, 0.9996589759984476)

### Feature Transformation

In [25]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [26]:
indexer = StringIndexer(inputCol='product_id',
                        outputCol='product_id_idx')
data_indexed = indexer.fit(data_sub).transform(data_sub)

indexer1 = StringIndexer(inputCol='customer_id',
                        outputCol='customer_id_idx')
data_indexed = indexer1.fit(data_indexed).transform(data_indexed)

In [27]:
data_indexed.show(5, truncate=True)

+----------+------+-----------+--------------+---------------+
|product_id|rating|customer_id|product_id_idx|customer_id_idx|
+----------+------+-----------+--------------+---------------+
|  10001012|   3.0|     709310|        2458.0|       216788.0|
|  10001012|   5.0|   10701688|        2458.0|        24473.0|
|  10001012|   5.0|   11763074|        2458.0|        26843.0|
|  10001012|   5.0|    9909549|        2458.0|         3377.0|
|  10001012|   5.0|    1827148|        2458.0|          396.0|
+----------+------+-----------+--------------+---------------+
only showing top 5 rows



### Train model

In [28]:
# Chia dữ liệu train_test
# Smaller dataset so we will use 0.8/0.2
(training, test)= data_indexed.randomSplit([0.8,0.2])

In [29]:
# xây dựng model
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=10,
          regParam=0.09,
          rank=25,
          userCol="customer_id_idx",
          itemCol="product_id_idx",
          ratingCol="rating",
          coldStartStrategy="drop",
          nonnegative=True)
model = als.fit(training)
# rank cho to hơn vì phim nhiều và user nhiều

In [30]:
data_indexed.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            361720|
|   mean| 4.474928121198717|
| stddev|1.0169193161968173|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



### Evaluate model

In [31]:
predictions = model.transform(test)

In [32]:
predictions.show(5)

+----------+------+-----------+--------------+---------------+----------+
|product_id|rating|customer_id|product_id_idx|customer_id_idx|prediction|
+----------+------+-----------+--------------+---------------+----------+
|   3524487|   5.0|   11354611|         148.0|        25895.0|  1.933549|
|   3524487|   5.0|   11174386|         148.0|         3561.0| 3.1160388|
|   3524487|   5.0|    1764742|         148.0|        15633.0| 2.5315573|
|   3524487|   5.0|    8267857|         148.0|         5571.0| 2.4736812|
|   3524487|   5.0|    6706179|         148.0|        51510.0| 1.8678849|
+----------+------+-----------+--------------+---------------+----------+
only showing top 5 rows



In [33]:
from pyspark.ml.evaluation import RegressionEvaluator

In [34]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean square error = "+ str(rmse))

Root-mean square error = 1.6666956469273468


### Tunning parameter

In [35]:
ALSExplicit = ALS( implicitPrefs=False, userCol="customer_id_idx", itemCol="product_id_idx", ratingCol="rating",
          coldStartStrategy="drop")

defaultModel = ALSExplicit.fit(training)

In [36]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

In [37]:
paramMapExplicit = ParamGridBuilder() \
                    .addGrid(ALSExplicit.rank, [30, 40]) \
                    .addGrid(ALSExplicit.maxIter, [5,10]) \
                    .addGrid(ALSExplicit.regParam, [0.1,0.01]) \
                    .addGrid(ALSExplicit.alpha, [2.0,3.0]) \
                    .build()

In [38]:
evaluatorR = RegressionEvaluator(metricName="rmse", labelCol="rating")

In [ ]:
CVALSExplicit = CrossValidator(estimator=ALSExplicit,
                            estimatorParamMaps=paramMapExplicit,
                            evaluator=evaluatorR,
                           numFolds=5)

CVModelEXplicit = CVALSExplicit.fit(training)

In [ ]:
predictions_t = CVModelEXplicit.transform(test)

In [ ]:
rmse_t = evaluator.evaluate(predictions_t)
print("Root-mean square error = "+ str(rmse_t))

Root-mean square error = 2.6181438290991856


In [ ]:
als_t = ALS(maxIter=10,
          regParam=0.1,
          rank = 30,
          userCol="customer_id_idx",
          itemCol="product_id_idx",
          ratingCol="rating",
          coldStartStrategy="drop",
          nonnegative=True)
model_t = als_t.fit(training)

In [ ]:
predictions_t = model_t.transform(test)

In [ ]:
rmse_t = evaluator.evaluate(predictions_t)
print("Root-mean square error = "+ str(rmse_t))

In [ ]:
# Nhận xét: 

### Make recommendations to all users

In [ ]:
# Get 5 recommendations which have highest rating
user_recs = model_t.recommendForAllUsers(5)

In [ ]:
user_recs.show(10,truncate=False)

+---------------+----------------------------------------------------------------------------------------------+
|customer_id_idx|recommendations                                                                               |
+---------------+----------------------------------------------------------------------------------------------+
|148            |[[2490, 8.144488], [3159, 7.861633], [3517, 7.857849], [3028, 7.841336], [3387, 7.765432]]    |
|463            |[[3517, 7.298908], [3735, 7.142155], [2490, 7.0662503], [3338, 7.048767], [3618, 6.976989]]   |
|471            |[[3028, 7.6680627], [4027, 7.54739], [3069, 7.5368056], [3766, 7.501582], [2810, 7.4372625]]  |
|496            |[[3075, 8.023589], [3516, 7.812709], [3009, 7.74424], [3028, 7.703732], [2471, 7.6857185]]    |
|833            |[[3387, 7.380689], [3870, 7.24335], [3167, 7.229719], [3315, 7.2231903], [3517, 7.1720223]]   |
|1088           |[[3028, 7.1429787], [2653, 7.1387944], [3618, 7.1260276], [3944, 7.0796223], [2

In [ ]:
user_recs.printSchema()

root
 |-- customer_id_idx: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- product_id_idx: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [ ]:
for user in user_recs.head(3):
    print(user)
    print("\n")

Row(customer_id_idx=148, recommendations=[Row(product_id_idx=2490, rating=8.144488334655762), Row(product_id_idx=3159, rating=7.861632823944092), Row(product_id_idx=3517, rating=7.85784912109375), Row(product_id_idx=3028, rating=7.841335773468018), Row(product_id_idx=3387, rating=7.765431880950928)])


Row(customer_id_idx=463, recommendations=[Row(product_id_idx=3517, rating=7.298908233642578), Row(product_id_idx=3735, rating=7.142155170440674), Row(product_id_idx=2490, rating=7.066250324249268), Row(product_id_idx=3338, rating=7.04876708984375), Row(product_id_idx=3618, rating=6.976988792419434)])


Row(customer_id_idx=471, recommendations=[Row(product_id_idx=3028, rating=7.668062686920166), Row(product_id_idx=4027, rating=7.547389984130859), Row(product_id_idx=3069, rating=7.536805629730225), Row(product_id_idx=3766, rating=7.501582145690918), Row(product_id_idx=2810, rating=7.437262535095215)])




### Save to file

In [ ]:
from time import time
t0 = time()

In [ ]:
df_reviewer_reviewer_id = data_indexed.select('customer_id_idx', 'customer_id').distinct()

In [ ]:
df_reviewer_reviewer_id.count()

251467

In [ ]:
df_reviewer_reviewer_id.show(5)

+---------------+-----------+
|customer_id_idx|customer_id|
+---------------+-----------+
|       156809.0|   18810996|
|        99015.0|   13554619|
|        45340.0|     386714|
|       195984.0|    5579748|
|        28390.0|   12467821|
+---------------+-----------+
only showing top 5 rows



In [ ]:
df_asin_asin_id = data_indexed.select('product_id_idx', 'product_id').distinct()

In [ ]:
df_asin_asin_id.count()

4218

In [ ]:
df_asin_asin_id.show(5)

+--------------+----------+
|product_id_idx|product_id|
+--------------+----------+
|        1162.0|  10793468|
|        1600.0|  11251743|
|        2216.0|  11308022|
|        3804.0|  11375305|
|        3237.0|  15350656|
+--------------+----------+
only showing top 5 rows



In [ ]:
new_user_recs = user_recs.join(df_reviewer_reviewer_id, on=['customer_id_idx'], how="left")

In [ ]:
new_user_recs.show(10, truncate=False)

+---------------+----------------------------------------------------------------------------------------------+-----------+
|customer_id_idx|recommendations                                                                               |customer_id|
+---------------+----------------------------------------------------------------------------------------------+-----------+
|148            |[[2490, 8.144488], [3159, 7.861633], [3517, 7.857849], [3028, 7.841336], [3387, 7.765432]]    |9528       |
|463            |[[3517, 7.298908], [3735, 7.142155], [2490, 7.0662503], [3338, 7.048767], [3618, 6.976989]]   |753408     |
|471            |[[3028, 7.6680627], [4027, 7.54739], [3069, 7.5368056], [3766, 7.501582], [2810, 7.4372625]]  |8232425    |
|496            |[[3075, 8.023589], [3516, 7.812709], [3009, 7.74424], [3028, 7.703732], [2471, 7.6857185]]    |10354187   |
|833            |[[3387, 7.380689], [3870, 7.24335], [3167, 7.229719], [3315, 7.2231903], [3517, 7.1720223]]   |61791      |


In [ ]:
new_user_recs.count()

211637

### Save to disk

In [ ]:
new_user_recs.write.parquet('Recommendation_U.parquet', mode='overwrite')
df_asin_asin_id.write.parquet('Recommendation_P.parquet', mode='overwrite')

In [ ]:
time_duration = time() - 10
print(time_duration)

1656701177.6223738


### Make recommendations to a particular user

In [ ]:
customer_id = "5917275"
find_user_rec = new_user_recs.filter(new_user_recs['customer_id'] == customer_id)
user = find_user_rec.first()

lst=[]
for row in user['recommendations']:
    row_f = df_asin_asin_id.filter(df_asin_asin_id.product_id_idx == row['product_id_idx'])
    row_f_first = row_f.first()
    lst.append((row['product_id_idx'], row_f_first['product_id'], row['rating']))
dic_user_rec = {'customer_id' : user.customer_id, 'recommendations':lst}

In [ ]:
dic_user_rec

{'customer_id': 5917275,
 'recommendations': [(3028, 654491, 7.4081244468688965),
  (2490, 53683384, 7.383239269256592),
  (2471, 30857999, 7.121313571929932),
  (3069, 24945084, 7.097879409790039),
  (2927, 52699359, 7.04839563369751)]}

### Đọc 2 file đã lưu để lấy dữ liệu đầu vào => Đề xuất

In [ ]:
new_user_recs = spark.read.parquet('Recommendation_U.parquet')

In [ ]:
new_user_recs.printSchema()

root
 |-- customer_id_idx: integer (nullable = true)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- product_id_idx: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)
 |-- customer_id: integer (nullable = true)



In [ ]:
new_user_recs.show(2)

+---------------+--------------------+-----------+
|customer_id_idx|     recommendations|customer_id|
+---------------+--------------------+-----------+
|            158|[[3238, 6.8577127...|   11909169|
|            199|[[3155, 7.4610467...|     101545|
+---------------+--------------------+-----------+
only showing top 2 rows



In [ ]:
df_asin_asin_id = spark.read.parquet('Recommendation_P.parquet')

In [ ]:
df_asin_asin_id.show(2)

+--------------+----------+
|product_id_idx|product_id|
+--------------+----------+
|        3052.0|  10138000|
|         550.0|  10857215|
+--------------+----------+
only showing top 2 rows



In [ ]:
customer_id = "5917275"
find_user_rec = new_user_recs.filter(new_user_recs['customer_id'] == customer_id)
find_user_rec.show(truncate=False)

+---------------+-----------------------------------------------------------------------------------------------+-----------+
|customer_id_idx|recommendations                                                                                |customer_id|
+---------------+-----------------------------------------------------------------------------------------------+-----------+
|1184           |[[3028, 7.4081244], [2490, 7.3832393], [2471, 7.1213136], [3069, 7.0978794], [2927, 7.0483956]]|5917275    |
+---------------+-----------------------------------------------------------------------------------------------+-----------+



In [ ]:
result = ''
for user in find_user_rec.collect():
    lst=[]
    for row in user['recommendations']:
        print(row)
        row_f = df_asin_asin_id.filter(df_asin_asin_id.product_id_idx == row['product_id_idx'])
        row_f_first = row_f.first()
        lst.append((row['product_id_idx'], row_f_first['product_id'], row['rating']))
    dic_user_rec = {'customer_id' : user.customer_id, 'recommendations':lst}
    result = dic_user_rec

Row(product_id_idx=3028, rating=7.4081244468688965)
Row(product_id_idx=2490, rating=7.383239269256592)
Row(product_id_idx=2471, rating=7.121313571929932)
Row(product_id_idx=3069, rating=7.097879409790039)
Row(product_id_idx=2927, rating=7.04839563369751)
